In [1]:
%cd openmeteopy-main

C:\Users\admin\JupyterNotebook_works\mtech-programing-labs\S2\Data-Mining-and-Big-Data\spark-work\project-deploy-on-pyspark\openmeteopy-main


In [4]:
import pickle
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType, StringType
import time
import pandas as pd

In [2]:
from openmeteo_py import OWmanager
from openmeteo_py.Hourly.HourlyForecast import HourlyForecast
from openmeteo_py.Daily.DailyForecast import DailyForecast
from openmeteo_py.Options.ForecastOptions import ForecastOptions
from openmeteo_py.Utils.constants import *

In [5]:
# Latitude, Longitude 
longitude = 76.399994
latitude =  9.400002

In [3]:
%cd ..

C:\Users\admin\JupyterNotebook_works\mtech-programing-labs\S2\Data-Mining-and-Big-Data\spark-work\project-deploy-on-pyspark


In [8]:
hourly = HourlyForecast()
daily = DailyForecast()
options = ForecastOptions(latitude,longitude,False,celsius,kmh,mm,iso8601,auto)

#notice that we had to give the value "None" for the hourly parameter,otherwise you'll be filling the hourly parameter instead of the daily one.
mgr = OWmanager(options,OWmanager.forecast, hourly.shortwave_radiation(), daily.all())
# Download data
meteo = mgr.get_data(1)

In [10]:
daily.rain_sum

<bound method DailyForecast.rain_sum of <openmeteo_py.Daily.DailyForecast.DailyForecast object at 0x00000269C05766C8>>

In [76]:
meteo['daily']['sunrise']

{'2023-05-30': '2023-05-30T06:01',
 '2023-05-31': '2023-05-31T06:01',
 '2023-06-01': '2023-06-01T06:02',
 '2023-06-02': '2023-06-02T06:02',
 '2023-06-03': '2023-06-03T06:02',
 '2023-06-04': '2023-06-04T06:02'}

In [6]:
def getdata(i):
    hourly = HourlyForecast()
    daily = DailyForecast()
    options = ForecastOptions(latitude,longitude,False,celsius,kmh,mm,iso8601,auto)

    #notice that we had to give the value "None" for the hourly parameter,otherwise you'll be filling the hourly parameter instead of the daily one.
    mgr = OWmanager(options,OWmanager.forecast, hourly.shortwave_radiation(), daily.all())
    # Download data
    meteo = mgr.get_data(1)
    
    test_forecast_X_df = {'time':[], 'temperature_2m_max':[], 'temperature_2m_min':[],
       'sunrise':[], 'sunset':[], 'precipitation_sum':[],
       'rain_sum':[0], 'snowfall_sum':[0], 'precipitation_hours':[],
       'windspeed_10m_max':[], 'windgusts_10m_max':[]}
    
    date = list(meteo['daily']['time'])[i]
    test_forecast_X_df['time'].append(date)
    test_forecast_X_df['temperature_2m_max'].append(meteo['daily']['temperature_2m_max'][date])
    test_forecast_X_df['temperature_2m_min'].append(meteo['daily']['temperature_2m_min'][date])
    test_forecast_X_df['sunrise'].append(meteo['daily']['sunrise'][date])
    test_forecast_X_df['sunset'].append(meteo['daily']['sunset'][date])
    test_forecast_X_df['precipitation_sum'].append(meteo['daily']['precipitation_sum'][date])
    test_forecast_X_df['precipitation_hours'].append(meteo['daily']['precipitation_hours'][date])
    test_forecast_X_df['windspeed_10m_max'].append( meteo['daily']['windspeed_10m_max'][date])
    test_forecast_X_df['windgusts_10m_max'].append(meteo['daily']['windgusts_10m_max'][date])
    
    df = pd.DataFrame(test_forecast_X_df)
    
    df['time_temp'] = df.time.str.split('-')
    df['Day'] = df.time_temp.str[2]
    df['Month'] = df.time_temp.str[1]
    df['Year'] = df.time_temp.str[0]

    df['Day'] = df.Day.astype('int')
    df['Month'] = df.Month.astype('int')
    df['Year'] = df.Year.astype('int')

    df.drop(['time_temp'], axis=1, inplace=True)
    
    df['sunrise temp'] = df['sunrise'].str.split('T')
    df['sunrise temp'] = df['sunrise temp'].str[1]
    df['sunrise temp'] = df['sunrise temp'].str.split(':')
    df['sunrise_hour'] = df['sunrise temp'].str[0]
    df['sunrise_minutes'] = df['sunrise temp'].str[1]

    df['sunrise_hour'] = df['sunrise_hour'].astype('int')
    df['sunrise_minutes'] = df['sunrise_minutes'].astype('int')

    df.drop(['sunrise temp'], axis=1, inplace=True)
    
    df['sunset temp'] = df['sunset'].str.split('T')
    df['sunset temp'] = df['sunset temp'].str[1]
    df['sunset temp'] = df['sunset temp'].str.split(':')

    df['sunset_hour'] = df['sunset temp'].str[0]
    df['sunset_minutes'] = df['sunset temp'].str[1]

    df['sunset_hour'] = df['sunset_hour'].astype('int')
    df['sunset_minutes'] = df['sunset_minutes'].astype('int')

    df.drop(['sunset temp'], axis=1, inplace=True)
    
    df_new = df.copy()
    df_new.drop(['time', 'sunrise', 'sunset'], axis=1, inplace=True)
    return date, df_new

In [23]:
%cd ..

C:\Users\admin\JupyterNotebook_works\mtech-programing-labs\S2\Data-Mining-and-Big-Data\spark-work\project-deploy-on-pyspark


In [44]:
df_new.shape

(1, 15)

In [33]:
df_new.columns

Index(['temperature_2m_max', 'temperature_2m_min', 'precipitation_sum',
       'rain_sum', 'snowfall_sum', 'precipitation_hours', 'windspeed_10m_max',
       'windgusts_10m_max', 'Day', 'Month', 'Year', 'sunrise_hour',
       'sunrise_minutes', 'sunset_hour', 'sunset_minutes'],
      dtype='object')

In [25]:
X_sc = load_enc.transform(df_new)
X_sc

array([[ 0.85759293,  0.70256966, -0.38050106, -0.76025093,  0.        ,
        -0.12874229,  0.44096296, -0.10053395,  2.35718533, -0.15865661,
         0.53305006, -1.96085854,  1.38782481, -0.17636354, -0.98122423]])

In [34]:
X_sc_df = pd.DataFrame(X_sc, columns=['temperature_2m_max', 'temperature_2m_min', 'precipitation_sum',
       'rain_sum', 'snowfall_sum', 'precipitation_hours', 'windspeed_10m_max',
       'windgusts_10m_max', 'Day', 'Month', 'Year', 'sunrise_hour',
       'sunrise_minutes', 'sunset_hour', 'sunset_minutes'])

In [35]:
X_sc_df

,temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,Day,Month,Year,sunrise_hour,sunrise_minutes,sunset_hour,sunset_minutes
0,0.857593,0.70257,-0.380501,-0.760251,0.0,-0.128742,0.440963,-0.100534,2.357185,-0.158657,0.53305,-1.960859,1.387825,-0.176364,-0.981224


In [7]:
# Create spark session
#from pyspark.conf import SparkConf
#conf = SparkConf()

In [7]:

#conf.set("spark.executor.heartbeatInterval", "3600s")

spark = SparkSession.builder.config().getOrCreate()

Py4JJavaError: An error occurred while calling o0.set.
: java.lang.NullPointerException: null key
	at org.apache.spark.SparkConf.set(SparkConf.scala:88)
	at org.apache.spark.SparkConf.set(SparkConf.scala:83)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.lang.Thread.run(Thre

In [82]:
sc = spark.sparkContext

In [83]:
# Unpickle, pkl file
model_rdd_pkl = sc.binaryFiles("flood_model.pkl")
model_rdd_data = model_rdd_pkl.collect()

In [84]:
# Load and broadcast python object over spark nodes
creditcardfrauddetection_model = pickle.loads(model_rdd_data[0][1])
broadcast_creditcardfrauddetection_model = sc.broadcast(creditcardfrauddetection_model)
print(broadcast_creditcardfrauddetection_model.value)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='mlogloss', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)


In [85]:
def predict(*cols):
    prediction = broadcast_creditcardfrauddetection_model.value.predict_proba((cols,))
    return float(prediction[0,1])

predict_udf = udf(predict, DoubleType())

In [86]:
feature_columns = X_sc_df.columns.to_list()

In [89]:
sparkDF = spark.createDataFrame(X_sc_df) 
sparkDF.printSchema()

AttributeError: 'SparkContext' object has no attribute 'createDataFrame'

In [88]:
# Create predictions using spark udf
sparkDF = sparkDF.withColumn("Prediction", predict_udf(*feature_columns))
sparkDF.show()

Py4JJavaError: An error occurred while calling o269.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 9) (10.10.18.54 executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(R

In [12]:
df = spark.read.csv('.csv', header=True)
df.show()

In [10]:
## loading scaler object
sc_file = 's_scaler.pkl'
with open(sc_file, 'rb') as file:
    load_enc = pickle.load(file)

In [11]:
model_file = 'flood_model.pkl'
with open(model_file, 'rb') as file:
    load_model = pickle.load(file)

In [14]:
i = 0
while True:
    date, X = getdata(i)
    X_sc = load_enc.transform(X)
    y_pred = load_model.predict(X_sc)
    print('Data: {} || Will be there flood(1->Yes or 0->No): {}'.format(X, y_pred[0]),'\n')
    time.sleep(30)
    i+=1
    if i==5:
        break

Data:    temperature_2m_max  temperature_2m_min  precipitation_sum  rain_sum  \
0                32.4                25.8                1.3         0   

   snowfall_sum  precipitation_hours  windspeed_10m_max  windgusts_10m_max  \
0             0                  8.0               14.9               31.7   

   Day  Month  Year  sunrise_hour  sunrise_minutes  sunset_hour  \
0   30      5  2023             6                1           18   

   sunset_minutes  
0              41   || Will be there flood(1->Yes or 0->No): 0 

Data:    temperature_2m_max  temperature_2m_min  precipitation_sum  rain_sum  \
0                31.8                26.8               10.4         0   

   snowfall_sum  precipitation_hours  windspeed_10m_max  windgusts_10m_max  \
0             0                 15.0               13.4               29.2   

   Day  Month  Year  sunrise_hour  sunrise_minutes  sunset_hour  \
0   31      5  2023             6                1           18   

   sunset_minutes  
0